# HistFactory

In [12]:
%%html
<p align="center"><iframe src="https://matthewfeickert.github.io/talk-SciPy-2020/index.html#7" width="1200" height="675"></iframe></p>

$$\large
f(n, a | \vec{\eta}, \vec{\chi} ) = \prod_{c \in \textrm{channels}} \prod_{b \in \textrm{bins}} \mathrm{Pois}(n_{cb} | \nu_{cb}(\vec{\eta}, \vec{\chi}))\, \prod_{\chi \in \vec{\chi}} c_{\chi}(a_{\chi} | \vec{\chi})
$$